# Before going to the main content ...
let's check if nvcc is available.<br>
If it shows a report on the Cuda compiler driver, then we are good to go!

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


# Hello world!

Let's try to run the hello-world program.<br>
Let me explain the code first.<br>
The code is located in this directory : ```codes/1_introduction/```<br>
It is a ```.cu``` file, which looks very similar to a ```.c``` file.<br>
However, some necessary changes have to be made.
- The function that runs in GPU is a ```__global__ void``` type.
- GPU functions ALWAYS have to be ```void``` type. (please confirm this!)
- The syntax for calling the GPU function is : ```func <<<num1, num2>>>();``` (I'll explain what these numbers are.)
- The output of the GPU is collected after synchronizing with the CPU by doing ```cudaDeviceSynchronize();```.<br>

The syntax for compiling and running is the following. <br>
```nvcc -o output_exe input_code -run```<br>

In this particular example, there are three parts.<br>

#### A function that runs on the CPU:
```
void helloCPU(){
    printf("Hello from the CPU.\n");
}
```
#### A function that runs on the GPU:

```
__global__ void helloGPU(){
    printf("Hello from the GPU.\n");
}
```
#### A main function:
```
int main(){
    helloCPU();
    helloGPU<<<1, 1>>>();
    cudaDeviceSynchronize();
}
```
Inside the main function, after compilation, the CPU runs the ```helloCPU()``` function just like a regular C program. Then the ```helloGPU()``` function is sent to block=1 and thread=1 of the GPU. So this only runs once. (This is explained in the next block.) After that, ```cudaDeviceSynchronize()``` makes sure that the GPU and the CPU are synchronized. Now, the output from the GPU is available to the CPU, so that "Hello from the GPU" can be printed on the screen.

In [2]:
!nvcc -o executables/hello codes/1_introduction/hello.cu -run

Hello from the CPU.
Hello from the GPU.


# CUDA Thread Hierarchy

CUDA stands for Compute Unified Device Architecture. A **thread** refers to the smallest unit of work that can be scheduled and executed by a GPU. These are organized into various levels of hierarchy to efficiently utilize the GPU's parallel processing capabilities. Thousands or even millions of threads can run simultaneously on a GPU, allowing for massive parallelism.
Threads within a block can cooperate and communicate with each other through shared memory.

Threads are grouped into **blocks**. A block is a logical unit that provides synchronization, communication, and memory sharing among its threads. You can think of it as a collection of threads working together. All threads within a block can access the same shared memory. The maximum number of threads per block depends on the GPU architecture.

A **grid** is a collection of blocks. Blocks within a grid can execute independently of each other, allowing for further parallelism. The blocks in a grid can be scheduled on any available multiprocessor (SM) within the GPU.

Now, let's come back to the syntax : ```someKernel<<<num1, num2>>>();```
Here, the GPU *kernel* is sent to the different threads from different blocks for parallel processing. The numbers here refers to the number of blocks and number of threads per block. i.e.,<br>
```someKernel<<<NUMBER_OF_BLOCKS, NUMBER_OF_THREADS_PER_BLOCK>>>();```<br>

**The kernel code is executed by every thread in every thread block configured when the kernel is launched**.<br>
- `someKernel<<< 1,  1>>>()` is configured to run in a single thread block which has a single thread and will therefore run only once.
- `someKernel<<< 1, 10>>>()` is configured to run in a single thread block which has 10 threads and will therefore run 10 times.
- `someKernel<<<10,  1>>>()` is configured to run in 10 thread blocks which each have a single thread and will therefore run 10 times.
- `someKernel<<<10, 10>>>()` is configured to run in 10 thread blocks which each have 10 threads and will therefore run 100 times.

Let's see this in action.

### Playing with number of threads and blocks (accelerating For loops):

The threads in each block and the blocks themselves are given indices (integers) which start from 0. These indices can be accessed through variables such as ```threadIdx.x``` and ```blockIdx.x```. The following is an example where I am printing out these indices for each thread. You can see that the print statements are 'not chronological'. They are being processed simultaneously in the threads.

In [3]:
!nvcc -o executables/printing_numbers codes/1_introduction/threads_and_blocks.cu -run

thread index (in each block) = 0, block index = 1 
thread index (in each block) = 1, block index = 1 
thread index (in each block) = 0, block index = 0 
thread index (in each block) = 1, block index = 0 


### Making the code idiot-proof
There is an issue with manually setting nBlocks and nThreads. The number of cores needed to execute the code is at least nBlocks * nThreadsPerBlock. That's why we can't just come up with some arbitrary number of blocks and thread per block. In my case, I have 896 cores in my GPU. So, the product, nBlocks * nThreadsPerBlock can't go beyond that.

CUDA Kernels have access to a special variable that gives the number of threads in a block: `blockDim.x`. Using this variable, in conjunction with `blockIdx.x` and `threadIdx.x`, we can find out a unique ID for each thread in all the blocks by calculating this : `threadIdx.x + blockIdx.x * blockDim.x`. The following is a detailed example demonstrating how it works.

Consider the execution configuration <<<10, 10>>> . It would launch a grid with a total of 100 threads, contained in 10 blocks of 10 threads. We would therefore hope for each thread to have the ability to calculate some index unique to itself between 0 and 99.

- If `blockIdx.x == 0`, then `blockIdx.x * blockDim.x` is 0. Adding to 0 the possible threadIdx.x values 0 through 9, then we can generate the indices 0 through 9 within the 100 thread grid.
- If `blockIdx.x == 1`, then `blockIdx.x * blockDim.x` is 10. Adding to 10 the possible threadIdx.x values 0 through 9, then we can generate the indices 10 through 19 within the 100 thread grid.
- If `blockIdx.x == 5`, then `blockIdx.x * blockDim.x` is 50. Adding to 50 the possible threadIdx.x values 0 through 9, then we can generate the indices 50 through 59 within the 100 thread grid.
- If `blockIdx.x == 9`, then `blockIdx.x * blockDim.x` is 90. Adding to 90 the possible threadIdx.x values 0 through 9, then we can generate the indices 90 through 99 within the 100 thread grid.

This is how we end up with unique IDs for each thread which runs from 0 to 99. Once we have that, we can control how many parallel processing happens in the GPU.

For example, first, we set how many cores to use.

`int N = 896;`<br>

This is the maxmium cores that I can use. I may choose a smaller number as well, depending on what I want to do. Assume that we have a desire to set threads_per_block exactly to 256. Then,<br>

`size_t threads_per_block = 256;`<br>

For a given number of cores to use, the number of blocks should be the following.<br> 

`size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;`

Inside the GPU function, we calculate the unique ID of the thread by doing the following.

`int idx = threadIdx.x + blockIdx.x * blockDim.x;`

Then, we tell the GPU to execute the job, only when `idx < N`.

Let's modify the previous example to include this unique ID. I am printing out the integers from 0 to 9, and hence choosing 10 threads for this operation. Also, I am choosing the number of threads per block to be 4. 

In [4]:
!nvcc -o executables/printing_numbers_2 codes/1_introduction/threads_and_blocks_2.cu -run

thread index (in each block) = 0 , block index = 2, thread ID = 8 
thread index (in each block) = 1 , block index = 2, thread ID = 9 
thread index (in each block) = 0 , block index = 0, thread ID = 0 
thread index (in each block) = 1 , block index = 0, thread ID = 1 
thread index (in each block) = 2 , block index = 0, thread ID = 2 
thread index (in each block) = 3 , block index = 0, thread ID = 3 
thread index (in each block) = 0 , block index = 1, thread ID = 4 
thread index (in each block) = 1 , block index = 1, thread ID = 5 
thread index (in each block) = 2 , block index = 1, thread ID = 6 
thread index (in each block) = 3 , block index = 1, thread ID = 7 


Notice that, the thread IDs are not organised from 0 to 9. This is because the blocks are not organised that way. You can see that the threads in each block are organised from 0 to 3 (except for the last block, where the thread id goes from 0 to 1, as the number of cores used here is exhausted). 

You can do a fun exercise of printing out the integers from 0 to 9. But instead of using a for loop in CPU, where the print statment is executed one after another, I am using the index features of the GPU to execute the print statement simulataneously.

I have two algorithms for doing that. The first one involves using 10 cores and printing out the unique thread id (which goes from 0 to 9). This is similar to the previous example. That's why I am not repreating it here. The second one involves a matrix of numbers {block index, thread index}, where only the numbers from the diagonal elements are being printed on screen. The GPU function will look like this.

```
__global__ void print_integers(){
    int block_index = blockIdx.x;
    int thread_index = threadIdx.x;
    if(block_index == thread_index){ //for only the diagonal elements in the matrix {block index, thread index}
        printf("%d", block_index);
    } 
}
```
Since we are printing out numbers from 0 to 9, both `block_index` and `thread_index` should have values from 0 to 9. That's why in the main function, we run the kernel as follows.
`print_integers<<<10, 10>>>();`

The first algorithm is better, because it prints out a number in each thread. In the second algorithm, it only prints out a number in some of the threads (where the block index matches the thread index). For a limited number of GPU cores, the first algorithm can print more numbers than the second one.

Try this example yourself. The disadvantage of this method is that, it is limited by the number of available cores. That's why its important to make the code 'idiot proof' by executing the processes only when the thread id is less than the number of avaliable cores.

# Memory allocation (pointers) in GPU

Instead of the memeory allocation tools from C, cuda has its own tools for allocating memory location to a pointer. Let's try an example where the GPU tries to double every element of an array. The array is just a set of integers from 0 to N. We will use a pointer, `*a` for the array. Here is the GPU function that doubles each element:
```
__global__
void doubleElements(int *a, int N){
  int i;
  i = blockIdx.x * blockDim.x + threadIdx.x; //This is a For loop equivalent.
  if (i < N){a[i] *= 2;}
}
```
In the CPU, we check whether this doubling has been properly done or not using the following.
```
bool checkElementsAreDoubled(int *a, int N){
  int i;
  for (i = 0; i < N; ++i){
    if (a[i] != i*2){
      return false;
      printf("%d", i);
    }
  }
  return true;
}
```
In the main function, we allocate memory locations to the pointer `*a` using `cudaMallocManaged(&a, size)` (instead of the traditional way : `a = (int *)malloc(size);`). This is how `*a` can be used on both the host (CPU) and the device (GPU). We also free the memory allocation by doing `cudaFree(a);` (instead of the traditional way : `free(a)`).<br>

This is the basic recipe for using pointers in GPU:
- define a pointer. In this case, `int *a;`
- Allocate a size to it : `cudaMallocManaged(&a, N*sizeof(int));` (since we are storing N integers)
- Do whatever you want with the pointer.
- Free the memeory allocation after all tasks are done : `cudaFree(a);`

Checkout the example - `codes/1_introduction/memory_allocation.cu`.<br>
If everything goes well, this example should print out TRUE.

In [3]:
!nvcc -o executables/memory_alloc codes/1_introduction/memory_allocation.cu -run

All elements were doubled? TRUE


# Dealing with datasets larger than the grid (grid-strides)

Consider an array with 1000 elements, and a grid with 250 threads (cores in GPU). In order to do 1000 parallel operations on the array, each thread in the grid will need to be used 4 times. One method to do this is to use something called a **grid-stride loop** within the kernel.

Consider a situation where `N` is the number of parallel tasks to be done. But the system does not have enough threads (`nThread < N`). In a grid-stride loop, each thread calculates its unique ID using `i = thread_ind + batch_ind * batch_dim`. Then, it performs the `i`-th operation. This is how the first `nThread` tasks are done. Now, each thread updates the ID to `i+nThread`, and performs the next `nThread` number of tasks. This keeps repeating until all the tasks are completed.

The following is a template of a kernel containing this grid-stride feature.
```
__global__ void kernel(int *a, int N){

  //First, calculate the ID of the kernel and get the grid-stride.
  int indexWithinTheGrid = threadIdx.x + blockIdx.x * blockDim.x;
  int gridStride = gridDim.x * blockDim.x;

  for (int i = indexWithinTheGrid; i < N; i += gridStride){
    //Note that, i takes values only upto N, because we want N parallel tasks
    //gridStride is the value by which i needs to 'jump'. 
    //Here, do work on a[i];
  }
}
```

Let's modify the previous example of doubling each element of an array. In this modified version, I am choosing the number of parallel tasks (`N`) to be 1000, which is bigger than the number of cores that I have. Check out the code here - `codes/1_introduction/grid-stride.cu`<br>
If it runs successfully, it should print out TRUE on the screen.

In [4]:
!nvcc -o executables/grid-stride codes/1_introduction/grid-stride.cu -run

All elements were doubled? TRUE


# Error handling

In case of an error, most CUDA functions return a value of type `cudaError_t`.  This can be used to check whether or not an error occurred while calling the function. Here is an example where error handling is performed for a call to `cudaMallocManaged`:

```
cudaError_t err = cudaMallocManaged(&a, N)
if (err != cudaSuccess){                          //`cudaSuccess` is provided by CUDA.
  printf("Error: %s\n", cudaGetErrorString(err)); //`cudaGetErrorString` is provided by CUDA.
}
```

However, kernels are defined to return `void` types, and not `cudaError_t` types. That's why, to check errors occuring at the time of kernel launch, cuda provides the `cudaGetLastError` funtion, which returns a value of the type `cudaError_t`. This is how it is used in the main code.
```
someKernel<<<1, -1>>>();  // -1 is not a valid number of threads. This is the error.

cudaError_t err = cudaGetLastError();
if (err != cudaSuccess){
  printf("Error: %s\n", cudaGetErrorString(err));
}
```

#### CUDA Error Handling Function
It can be helpful to create a macro that wraps CUDA function calls for checking errors. Here is an example.

```
#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result){
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

int main(){
 //blah blah blah

 //The macro can be wrapped around any function returning
 //a value of type `cudaError_t`.

  checkCuda( cudaDeviceSynchronize() )
}
```

#### An exercise:
The following is a modified example of 'doubling the integer array'. I have intentionally put two kinds of errors here.
- Error during kernel launch : setting the number of threads per block beyond 1024.
- Error in the kernel itself : trying to access elements of a beyond its range (asynchronous error)
The code is here - `codes/1_introduction/error_handling.cu`
The errors are caught in `cudaError_t` type variables and displayed on screen.

In [11]:
!nvcc -o executables/errors codes/1_introduction/error_handling.cu -run

Error: invalid configuration argument
All elements were doubled? FALSE
